In [100]:
from ollama import generate, chat, pull
from pydantic import BaseModel
from typing import List

pull('gemma3:12b')

ProgressResponse(status='success', completed=None, total=None, digest=None)

In [68]:
class QnA(BaseModel):
    question: str
    answer: bool
    explanation: str

class Response(BaseModel):
    responses: List[QnA] = []

In [101]:

response = generate(
    system='''
    Given a top view image or street view images, you are going to roughly estimate house conditions. 
    Your answer should be based only on your observation. 
    The format of your response must include question, answer (yes, no, maybe), explanation (within 50 words) for each question.
    ''',
    prompt='Is there any damage on the roof?',
    model='gemma3:12b',
    images=['docs/data/test1.jpg'],
    format=Response.model_json_schema(),
    options={
                "temperature":0,
                "top_k":0.8,
                "top_p":0.8
            }
)

# Parse response content into the Pydantic model
# res = Response.model_validate_json(response.message.content)  # Note: use ['response'] for generate()
# res


In [63]:
response.response

"\nBased on the image provided, it is not possible to conclusively determine if there is any damage on the roof due to the lack of detail and resolution. The image does not allow for a clear view of the roof's condition, which would be necessary to provide an accurate assessment. A closer inspection or additional images from different angles would be required to accurately answer this question."

In [30]:
res = Response.model_validate_json(response.response)

In [50]:
res

Response(responses=[QnA(question='Is there any damage on the roof?', answer='no', explanation='The roof appears to be intact, with no visible signs of damage or wear. The shingles are evenly spaced and securely fastened.')])

In [104]:
system='''
    Given a top view image or street view images, you are going to roughly estimate house conditions. 
    Your answer should be based only on your observation. 
    The format of your response must include question, answer (yes=true, no=false), explanation (within 50 words) for each question.
    '''
prompt='Is there any physical damage on the roof?'
response = chat(
    
    model='gemma3:12b',
    messages=[
                {
                    'role': 'system',
                    'content': system
                },
                {
                    'role': 'user',
                    'content': prompt,
                    'images': ['docs/data/test3.jpg']
                }
            ],
    format=Response.model_json_schema(),
    options={
                "temperature":0,
                "top_k":0.8,
                "top_p":0.8
            }
)

res2 = Response.model_validate_json(response.message.content)

In [105]:
res2

Response(responses=[QnA(question='Is there any physical damage on the roof?', answer=False, explanation='Based on the image, the roof appears to be intact and without any visible signs of damage like missing shingles or structural issues. It looks generally well-maintained.')])

In [2]:
import urbanworm

In [3]:
from urbanworm import UrbanDataSet

In [4]:
bbox = (-83.235572,42.348092,-83.235154,42.348806)
data = UrbanDataSet()
data.bbox2Buildings(bbox)

'5 buildings found in the bounding box.'

In [80]:
system = '''
    Given a top view image, you are going to roughly estimate house conditions. Your answer should be based only on your observation. 
    The format of your response must include question, answer (yes=1 or no=0), explaination (within 50 words)
'''
prompt = {
    'top':'''
        Is there any damage on the roof?
    '''
}

# inspect the aerial images only (with type='top')
res = data.loopUnitChat(system=system, prompt=prompt, type='top', epsg=2253, model='gemma3:12b')

Processing...: 100%|█████████████████████████| 5/5 [01:34<00:00, 18.91s/it]


In [81]:
import geopandas as gpd
df = gpd.GeoDataFrame(data.to_gdf(), geometry="geometry")

In [82]:
df.pop('top_view_base64')
df.to_geo_dict()

{'type': 'FeatureCollection',
 'features': [{'id': '0',
   'type': 'Feature',
   'properties': {'top_view_question1': 'Is there any damage on the roof?',
    'top_view_answer1': '0',
    'top_view_explanation1': "The roof appears intact and doesn't show any obvious signs of damage like missing shingles or structural issues. The image quality is limited, but no immediate concerns are visible."},
   'geometry': {'type': 'Point',
    'coordinates': (-83.23528425002904, 42.34874838775614)}},
  {'id': '1',
   'type': 'Feature',
   'properties': {'top_view_question1': 'Is there any damage on the roof?',
    'top_view_answer1': '0',
    'top_view_explanation1': 'The roof appears relatively intact from this top-down view. There are no obvious signs of missing shingles, holes, or other structural damage visible.'},
   'geometry': {'type': 'Point',
    'coordinates': (-83.23527843983697, 42.34859720064423)}},
  {'id': '2',
   'type': 'Feature',
   'properties': {'top_view_question1': 'Is there a

In [83]:
import json
json.dumps(df.to_geo_dict())

'{"type": "FeatureCollection", "features": [{"id": "0", "type": "Feature", "properties": {"top_view_question1": "Is there any damage on the roof?", "top_view_answer1": "0", "top_view_explanation1": "The roof appears intact and doesn\'t show any obvious signs of damage like missing shingles or structural issues. The image quality is limited, but no immediate concerns are visible."}, "geometry": {"type": "Point", "coordinates": [-83.23528425002904, 42.34874838775614]}}, {"id": "1", "type": "Feature", "properties": {"top_view_question1": "Is there any damage on the roof?", "top_view_answer1": "0", "top_view_explanation1": "The roof appears relatively intact from this top-down view. There are no obvious signs of missing shingles, holes, or other structural damage visible."}, "geometry": {"type": "Point", "coordinates": [-83.23527843983697, 42.34859720064423]}}, {"id": "2", "type": "Feature", "properties": {"top_view_question1": "Is there any damage on the roof?", "top_view_answer1": "0", "

In [ ]:
import ollama
response = ollama.chat(
    model='gemma3',
    messages=[
        {
            'role': 'system',
            'content': 'you are data spatial data analyst'
        },
        {
            'role': 'user',
            'content': f'please summarize the data: {json.dumps(df.to_geo_dict())}',
        }
    ],
    options={
        "temperature":0.3,
        "top_k":0.8,
        "top_p":0.8
    },
    stream=True
)
for chunk in response:
    print(chunk['message']['content'], end='', flush=True)

Okay, here's a summary of the data, presented from a spatial data analysis perspective:

**Overall Assessment:**

This data represents a series of top-down visual inspections of roofs, conducted at four distinct locations.  The consistent finding across all inspections is that **no immediate damage to the roofs is detected** based on these top-down views.

**Spatial Analysis Highlights:**

* **Consistent Findings:** All four features (representing individual roof inspections) report a negative response to the question: "Is there any damage on the roof?"  This suggests a consistent observation across the surveyed locations.
* **Geographic Clustering:** The four points are clustered geographically. All are located very close to each other, suggesting they are inspecting properties in a relatively small area.  This is important for understanding the scope of the inspection.
* **Point Locations:** The coordinates indicate the precise location of each inspection.  This allows for mapping an

In [87]:
for i in res:
    print(i['message']['content'], end='', flush=True)